## データの取得・確認

In [1]:
import sys
sys.path.append(r'C:/Users/koki5/Dropbox/Jupyter/ScratchML')

import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn import model_selection
from termcolor import cprint

from common.module.evaluation import score


cancer = datasets.load_breast_cancer()
cancer_data = pd.DataFrame(data=cancer.data, columns=cancer.feature_names)
cancer_target = pd.Series(data=cancer.target, name='target')

X_train, X_test, y_train, y_test = model_selection.train_test_split(cancer_data, cancer_target)

cprint('Cancer Data (samples: {}, features: {})'.format(*cancer_data.shape), 'blue', attrs=['bold'])
cprint('Cancer Target (samples: {})'.format(*cancer_target.shape), 'blue', attrs=['bold'])

Cancer Data (samples: 569, features: 30)
Cancer Target (samples: 569)


In [2]:
cancer_data.head(3)

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.8,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.6,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.9,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.8,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.0,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.5,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758


In [3]:
cancer_target.unique()

array([0, 1], dtype=int64)

## データ前処理

In [4]:
def standardize(X):
    """標準化する.
    
    X: データセット
    """
    return (X - np.mean(X)) / np.std(X)


def add_ones(X):
    """特徴量の0列目に1を加える.
    
    X: データセット
    """
    X['0'] = 1
    return X.sort_index(axis=1, ascending=True)


X_train_std = standardize(X_train)
X_test_std = standardize(X_test)

# バイアス項を追加
X_train_std = add_ones(X_train_std)
X_test_std = add_ones(X_test_std)

X_train_std.head(3)

,0,area error,compactness error,concave points error,concavity error,fractal dimension error,mean area,mean compactness,mean concave points,mean concavity,...,worst area,worst compactness,worst concave points,worst concavity,worst fractal dimension,worst perimeter,worst radius,worst smoothness,worst symmetry,worst texture
155,1,-0.489176,-0.578268,-0.646886,-0.441796,-0.549499,-0.526900,-0.739532,-0.659755,-0.635642,...,-0.537623,-0.506190,-0.499802,-0.382754,-0.190608,-0.598685,-0.534452,-0.504332,0.311731,-0.084383
406,1,-0.376679,-0.752531,-0.511638,-0.502048,-0.896513,0.436505,-0.395377,-0.095895,-0.432883,...,0.155384,-0.547460,-0.033142,-0.210360,-0.798411,0.286973,0.337877,-0.551201,-0.209953,-1.008287
236,1,2.456249,0.139186,0.870911,0.483431,-0.320703,2.903865,1.175989,1.916754,1.324780,...,3.760577,0.955786,2.185725,1.438418,0.105153,3.010448,3.153889,0.620505,0.296158,1.437438


## モデルの構築・訓練

In [5]:
class LogisticRegression(object):
    """勾配降下法に基づくロジスティック回帰."""
    
    
    def __init__(self, alpha=0.01, eps=1e-6):
        """
        ---パラメータ-------
        alpha: 学習率
        eps: 収束の目安
        --------------------
        
        ---属性-------------
        _w: パラメータ（重み）
        --------------------
        """
        self.alpha = alpha
        self.eps = eps
        
        self._w = 0
        
        
    def fit(self, X, y):
        """訓練データで学習する.
        
        ---パラメータ-------
        X: 訓練データセット
        y: ターゲット
        --------------------
        
        """
        # パラメータの初期値はランダムに設定する
        theta = np.random.rand(X.shape[1])
        error = self.J(X, y, theta)
        count = 0
        diff = 1
        
        # コスト関数の値が殆ど変動しなくなったら収束とする
        while diff > self.eps:
            # パラメータの更新処理
            grad = (1 / len(y)) * np.dot(self.h(X, theta) - y, X)
            theta = theta - self.alpha * grad
            
            # 収束状況を確認するための処理
            current_error = self.J(X, y, theta)
            diff = error - current_error
            error = current_error
            count += 1
            print('\r【{}回目】Error: {}, Diff: {}, Grad: {}'.format(count, error, diff, grad.sum()), end='')
            
        self._w = theta
        return self
    
    
    def predict(self, X):
        """クラスラベルを予測する.
        
        X: テストデータセット
        """
        return np.where(self.h(X, self._w) >= 0.5, 1, 0)
        
    
    def activate(self, z):
        """ロジスティック関数（活性化関数）.
        
        z: 仮説関数の予測値
        """
        return 1 / (1 + np.exp(-z))
    

    def h(self, X, theta):
        """仮説関数.
        
        X: 訓練データセット
        theta: パラメータ（重み）
        """
        return self.activate(np.dot(X, theta))
    

    def J(self, X, y, theta):
        """目的関数.
        
        X: 訓練データセット
        y: ターゲット
        theta: パラメータ（重み）
        """
        delta = 1e-7 # np.log()に0が混在しないようにする
        return - (1 / len(y)) * (np.sum(y * np.log(self.h(X, theta) + delta) + (1 - y) * np.log(1 - self.h(X, theta) + delta)))

In [6]:
lr = LogisticRegression()
lr.fit(X_train_std, y_train)

【112回目】Error: 0.9008360723964619, Diff: 0.025686750812016768, Grad: 8.443084929072047

【227回目】Error: 0.2398692936337165, Diff: 0.0009755926089455857, Grad: 1.6319569954202686

【336回目】Error: 0.18222984437077888, Diff: 0.00029964624942582785, Grad: 0.845182122671486

【451回目】Error: 0.15763740680442115, Diff: 0.00015723536927098047, Grad: 0.5593355100588054

【559回目】Error: 0.14374565096998967, Diff: 0.00010664399657503432, Grad: 0.4279786181801992

...

【780回目】Error: 0.1258611744936102, Diff: 6.245984066893806e-05, Grad: 0.29598544947042243

【886回目】Error: 0.1198659765166602, Diff: 5.137999445591068e-05, Grad: 0.2602385613535106

【988回目】Error: 0.11504772377039933, Diff: 4.353698691633712e-05, Grad: 0.23407719852030603

【1100回目】Error: 0.1105577005842464, Diff: 3.699585428411367e-05, Grad: 0.21148175417205284

【12260回目】Error: 0.05402013721594668, Diff: 9.99989878069929e-07, Grad: 0.024211643412183147875

## 予測・評価

In [7]:
predict = pd.DataFrame(lr.predict(X_train_std), columns=['TARGET'])
predict.head(10)

,TARGET
0,1
1,1
2,0
3,1
4,0
5,0
6,0
7,0
8,0
9,1


In [8]:
train_score = score(predict, y_train)
test_score = score(pd.DataFrame(lr.predict(X_test_std), columns=['target']), y_test)
cprint('train_score: {}\ntest_score: {}'.format(train_score, test_score), 'red', attrs=['bold'])

train_score: 0.9859154929577465
test_score: 0.9790209790209791
